# Project 2.2 Question 1
### Chunlei Zhou

## Define Model and Develop Helper Functions

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import unicodedata
import string
import torch
import torch.nn as nn
import random
import numpy as np

In [2]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [3]:
def findFiles(path):
    return glob.glob(path)

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters)

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles('data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

def findName(dict, name):
    keys = dict.keys()
    for key in keys:
        if name in dict[key]:
            return key
    return ''

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Turn a line into a <line_length x 1 x n_letters>, or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

# Interpret output
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

# Get a training example (a name and its language):
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()
    rnn.zero_grad()
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
    loss = criterion(output, category_tensor)
    loss.backward()
    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)
    return output, loss.item()

# Just return an output given a line
def evaluate(line_tensor):
    hidden = rnn.initHidden()
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
    return output

# Construct a Data Frame
def data_frame(category_lines):
    data_frame = [[],[]]
    for key in all_categories:
        for value in category_lines[key]:
            data_frame[0].append(value)
            data_frame[1].append(key)
    return data_frame

## 1. Systematic processing of data

In [4]:
criterion = nn.NLLLoss()
learning_rate = 0.005
n_iters = 100000
n_hiddens = [32, 64, 128]
DF = data_frame(category_lines)
total_samples = len(DF[0])
randomize_order = np.arange(0, total_samples)
n_epoch = 5

### 1.1 Effect of hidden state length
Report the accuracy numbers yielded by the different algorithm variants.

In [5]:
print('======= 1.1 Accuracy Report =======')
for n_hidden in n_hiddens:
    rnn = RNN(n_letters, n_hidden, n_categories)
    current_loss = 0
    all_losses = []
    for iter in range(1, n_iters + 1):
        category, line, category_tensor, line_tensor = randomTrainingExample()
        output, loss = train(category_tensor, line_tensor)
        current_loss += loss
    confusion = torch.zeros(n_categories, n_categories)
    n_confusion = total_samples
    for i in range(n_confusion):
        category = DF[1][i]
        line = DF[0][i]
        category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
        line_tensor = lineToTensor(line)
        output = evaluate(line_tensor)
        guess, guess_i = categoryFromOutput(output)
        category_i = all_categories.index(category)
        confusion[category_i][guess_i] += 1
    accuracy = sum(confusion.diag()) / sum(sum(confusion))
    print('n_hidden =', str(n_hidden)+':', 'Accuracy is %f' % accuracy.item())

======= 1.1 Accuracy Report =======
n_hidden = 32: Accuracy is 0.510461
n_hidden = 64: Accuracy is 0.553452
n_hidden = 128: Accuracy is 0.596941


### 1.2 Effect of systematic training
Report the accuracy numbers yielded by the algorithm thusly trained and evaluated for the different values of hidden state size.

In [ ]:
print('======= 1.2 Accuracy Report =======')
for n_hidden in n_hiddens:
    rnn = RNN(n_letters, n_hidden, n_categories)
    current_loss = 0
    all_losses = []
    for epoch in range(n_epoch):
        np.random.shuffle(randomize_order)
        for order in randomize_order:
            category = DF[1][order]
            line = DF[0][order]
            category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
            line_tensor = lineToTensor(line)
            output, loss = train(category_tensor, line_tensor)
            current_loss += loss
    confusion = torch.zeros(n_categories, n_categories)
    n_confusion = total_samples
    for i in range(n_confusion):
        category = DF[1][i]
        line = DF[0][i]
        category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
        line_tensor = lineToTensor(line)
        output = evaluate(line_tensor)
        guess, guess_i = categoryFromOutput(output)
        category_i = all_categories.index(category)
        confusion[category_i][guess_i] += 1
    accuracy = sum(confusion.diag()) / sum(sum(confusion))
    print('n_hidden =', str(n_hidden)+':', 'Accuracy is %f' % accuracy.item())

======= 1.2 Accuracy Report =======
n_hidden = 32: Accuracy is 0.727658
n_hidden = 64: Accuracy is 0.745940
